# <center>Estimating network structure from unreliable measurements</center>

In [1]:
import numpy as np
import networkx as nx
import random as random
from operator import itemgetter
from scipy.special import binom

## 1. Erdös-Rényi with independent edge measurements : undirected case

### 1.1 Initialization

Number of nodes n and list of all node pairs.

In [29]:
n = 50
node_pairs = list()
for i in range(n):
    for j in range(i,n):
        if i != j:
            node_pairs.append((i,j))

Graph parameters $\gamma$.

In [30]:
w = 0.05 # prob of an edge between two nodes
print("w = {}".format(w))

w = 0.05


Generate graph.

In [31]:
G = nx.gnp_random_graph(n,w)
print("nb edges : ", G.number_of_edges())

nb edges :  74


Data parameters $\theta$.

In [74]:
a = 0.7 # true positive rate
b = 0.3 # false positive rate

Generate data.

In [75]:
n_obs = 100 # nb of times we observe each node pair
N = {e:n_obs for e in node_pairs} # nb obs for each node pair
E = {e:0 for e in node_pairs} # nb edges seen during observations for each node pair

for k in range(n_obs):
    for e in node_pairs:
        dice = random.random()
        # if true positive
        if e in G.edges and dice < a:
            E[e] += 1
        # if false positive
        elif e not in G.edges and dice < b:
            E[e] += 1

In [76]:
for e in G.edges:
    print(E[e])
print()
for e in node_pairs:
    if e not in G.edges:
        print(E[e])

69
71
65
77
75
64
77
67
69
69
67
67
73
66
78
74
71
71
65
73
62
73
72
72
65
70
63
77
73
76
72
64
67
72
68
72
63
82
73
75
62
77
74
67
64
71
69
70
77
74
72
63
73
66
66
67
64
68
80
76
73
71
66
67
66
67
66
66
77
69
72
61
68
72

35
28
24
32
30
25
29
36
26
28
27
25
37
23
40
25
36
25
26
23
27
33
27
38
28
25
31
29
30
28
24
32
36
33
26
30
33
29
34
29
28
40
27
27
30
30
27
20
40
26
26
33
37
27
27
28
25
24
36
30
23
35
28
31
34
35
25
30
33
33
34
35
33
31
33
30
29
31
32
31
40
32
34
39
27
23
31
27
30
27
24
29
42
30
27
29
29
36
27
37
37
24
22
33
29
30
38
34
37
32
31
32
31
31
31
28
31
31
30
27
32
34
27
31
34
36
25
34
32
31
27
30
30
36
35
23
28
28
37
35
27
27
34
33
28
26
28
23
32
28
23
27
34
34
34
30
27
37
36
33
30
32
27
23
34
25
34
29
26
32
23
37
31
26
35
38
31
30
32
36
24
33
28
33
25
30
27
28
30
33
33
31
32
31
30
37
24
22
36
29
33
25
33
32
33
32
32
25
25
33
30
34
28
32
20
29
35
28
28
24
26
30
32
36
33
29
24
29
29
29
31
33
35
26
21
22
30
24
29
28
24
31
27
31
27
35
26
33
26
33
24
27
29
31
21
39
32
26
33


### 1.2 Iterations

Choose number of repetitions of the whole algorithm and max iterations for each repetition.

In [77]:
repetitions = 100
max_iter = 1000

Proceed.

In [80]:
# save values at each big iter
A = list()
B = list()
W = list()


# we do multiple repetitions and then take the mean
for k in range(repetitions):
    
    try:
        
        # random initialization of the parameters
        w = random.random()
        a = random.random()
        b = random.random()

        # iter
        for l in range(max_iter):

            old_w, old_a, old_b = w, a, b

            # compute Qij
            Q = dict()
            for (i,j) in N:
                Q[i,j] = w * a**E[i,j] * (1-a)**(N[i,j]-E[i,j])
                Q[i,j] /= w * a**E[i,j] * (1-a)**(N[i,j]-E[i,j]) + (1-w) * b**E[i,j] * (1-b)**(N[i,j]-E[i,j])

            # update w,a,b
            w = sum(Q.values()) / binom(n,2)
            numerator_a, numerator_b = 0, 0
            denominator_a, denominator_b = 0, 0
            for (i,j) in N:
                numerator_a += Q[i,j] * E[i,j]
                numerator_b += (1-Q[i,j]) * E[i,j]
                denominator_a += Q[i,j] * N[i,j]
                denominator_b += (1-Q[i,j]) * N[i,j]
            a = numerator_a / denominator_a
            b = numerator_b / denominator_b

            # break if no sufficient evolution
            if np.abs(min([a-old_a, b-old_b, w-old_w])) < 0.001:
                break

        # save a,b,w
        A.append(a)
        B.append(b)
        W.append(w)
    
    except:
        continue
    
    
# print values
w_final = max([(w, W.count(w)/len(W)) for w in set(W)], key=itemgetter(1))
a_final = max([(a, A.count(a)/len(A)) for a in set(A)], key=itemgetter(1))
b_final = max([(b, B.count(b)/len(B)) for b in set(B)], key=itemgetter(1))
print(" w = {}\n a = {}\n b = {}\n".format(w_final, a_final, b_final))

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars


 w = (0.939591818258402, 0.01)
 a = (0.699999922170325, 0.01)
 b = (0.6625330533080851, 0.01)



Probability of the real graph.

In [81]:
qA = 1
for (i,j) in N:
    if (i,j) in G.edges:
        qA *= Q[i,j]
    else:
        qA *= 1 - Q[i,j]
print(qA)

0.9999764667266384


Probability of other graphs selected at random.

In [89]:
for k in range(1000):
    G_test = nx.gnp_random_graph(n,w)
    qA = 1
    for (i,j) in N:
        if (i,j) in G_test.edges:
            qA *= Q[i,j]
        else:
            qA *= 1 - Q[i,j]
    if qA != 0:
        print(qA)

## 2. Erdös-Rényi with independent edge measurements : directed case

### 1.1 Initialization

Number of nodes n and list of all node pairs.

In [5]:
n = 20
node_pairs = list()
for i in range(n):
    for j in range(n):
        if i != j:
            node_pairs.append((i,j))

Graph parameters $\gamma$.

In [6]:
w = 0.05 # prob of an edge between two nodes
print("w = {}".format(w))

w = 0.05


Generate graph.

In [7]:
G = nx.gnp_random_graph(n, w, directed=True)
print("nb edges : ", G.number_of_edges())

nb edges :  18


Data parameters $\theta$.

In [8]:
a = 0.7 # true positive rate
b = 0.3 # false positive rate

Generate data.

In [9]:
n_obs = 50 # nb of times we observe each node pair
N = {e:n_obs for e in node_pairs} # nb obs for each node pair
E = {e:0 for e in node_pairs} # nb edges seen during observations for each node pair

for k in range(n_obs):
    for e in node_pairs:
        dice = random.random()
        # if true positive
        if e in G.edges and dice < a:
            E[e] += 1
        # if false positive
        elif e not in G.edges and dice < b:
            E[e] += 1

In [10]:
for e in G.edges:
    print(E[e])
print()
print("--------------------------------------------")
print()
for e in node_pairs:
    if e not in G.edges:
        print(E[e])

37
36
41
34
31
36
32
36
39
34
34
34
38
37
40
37
34
32

--------------------------------------------

11
16
13
17
13
9
13
12
13
16
14
10
10
10
13
21
16
19
22
21
12
15
19
14
22
14
14
19
12
17
17
16
15
18
15
12
15
14
12
12
17
16
14
14
15
16
16
14
16
13
20
17
25
20
18
21
12
16
15
14
18
13
19
16
13
18
18
15
16
11
7
15
18
17
18
10
17
15
13
19
16
15
17
14
14
13
18
14
17
19
11
9
19
14
12
15
12
18
13
20
14
13
15
17
14
19
13
15
19
14
15
15
10
18
16
18
17
17
14
18
17
14
19
12
18
12
6
15
13
19
18
21
20
12
8
17
12
13
14
18
9
9
12
16
13
13
9
14
19
18
13
15
13
17
17
13
15
24
12
12
13
15
16
15
8
17
14
16
15
18
16
18
17
17
12
9
9
18
15
17
10
14
21
18
16
19
12
20
19
14
16
15
13
12
14
17
26
13
13
15
18
17
9
16
14
11
12
19
15
17
15
16
17
18
12
16
18
13
21
23
11
10
23
21
13
13
16
16
15
14
20
12
19
14
12
12
16
16
12
14
13
14
14
14
16
19
23
13
25
15
14
8
13
11
19
21
19
16
12
11
9
11
12
15
14
13
14
15
22
14
11
20
18
16
14
12
11
19
13
12
13
14
20
19
14
16
13
20
18
18
12
12
15
16
14
12
17
9
19
11
19
17
11
14
12

### 1.2 Iterations

Choose max iterations.

In [11]:
max_iter = 100

Proceed.

In [12]:

# random initialization of the parameters
w = random.random()
a = random.random()
b = random.random()

# iter
for l in range(max_iter):

    old_w, old_a, old_b = w, a, b

    # compute Qij
    Q = dict()
    for (i,j) in N:
        Q[i,j] = w * a**E[i,j] * (1-a)**(N[i,j]-E[i,j])
        Q[i,j] /= w * a**E[i,j] * (1-a)**(N[i,j]-E[i,j]) + (1-w) * b**E[i,j] * (1-b)**(N[i,j]-E[i,j])

    # update w,a,b
    w = sum(Q.values()) / (n*(n-1))
    numerator_a, numerator_b = 0, 0
    denominator_a, denominator_b = 0, 0
    for (i,j) in N:
        numerator_a += Q[i,j] * E[i,j]
        numerator_b += (1-Q[i,j]) * E[i,j]
        denominator_a += Q[i,j] * N[i,j]
        denominator_b += (1-Q[i,j]) * N[i,j]
    a = numerator_a / denominator_a
    b = numerator_b / denominator_b

    # break if no sufficient evolution
    if np.abs(min([a-old_a, b-old_b, w-old_w])) < 0.001:
        break

# print result
print(" w = {}\n a = {}\n b = {}\n".format(w, a, b))

 w = 0.04787682021357658
 a = 0.7111904890083998
 b = 0.30204289305509435



Probability of the real graph.

In [13]:
qA = 1
for (i,j) in N:
    if (i,j) in G.edges:
        qA *= Q[i,j]
    else:
        qA *= 1 - Q[i,j]
print(qA)

0.8132002649041087


Probability of other graphs selected at random.

In [14]:
for k in range(100):
    G_test = nx.gnp_random_graph(n, w, directed=True)
    qA = 1
    for (i,j) in N:
        if (i,j) in G_test.edges:
            qA *= Q[i,j]
        else:
            qA *= 1 - Q[i,j]
    if qA != 0:
        print(qA)

8.076012262882793e-227
5.236291648480574e-263
5.97728950105169e-226
1.705670817474842e-249
6.584408371631693e-229
9.223388558580706e-227
3.3514386855899993e-278
6.047637341628052e-292
1.5305655488234576e-258
6.1211023e-317
4.4738357915697135e-254
7.699899034872014e-242
1.423401027565846e-305
1.45748e-318
1.0274751378274406e-258
5.361651265466131e-276
4.807062421164956e-289
5.109443209255372e-254
3.154277726463378e-272
2.223923398965485e-290
1.8765911912961e-297
3.6457491315276285e-219
3.642598143901753e-223
1.105373451592153e-248
1.0034497667527208e-245
2.8634360541162268e-269
1.7263907905561049e-274
1.8122852908106998e-255
3.232586711109272e-281
2.0930927714024647e-284
7.163460000409805e-248
6.191699800896553e-227
2.570135927297599e-159
1.6053283344730708e-243
1.2324388008152681e-276
2.8609549727189466e-273
1.9222325558436766e-269
2.3345720365761547e-271
2.4783526878472482e-281
7.790520122139732e-308
3.154277746198577e-272
1.923899563285213e-265
4.4206560660902565e-307
2.8993046385686